In [1]:
import sqlite3
import pandas as pd

# Создание соединения с базой данных
connection = sqlite3.connect('../datasets/checking-logs.sqlite')

# Создание таблицы datamart
query = '''
SELECT c.uid, c.labname, c.timestamp as first_commit_ts, 
                         MIN(p.datetime) as first_view_ts 
                         FROM checker as c 
                         LEFT JOIN pageviews as p 
                         ON c.uid = p.uid
                         WHERE c.status = 'ready'
                              and c.numTrials = 1 
                              and c.labname in ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1') 
                              and c.uid LIKE 'user_%' 
                         GROUP by c.timestamp, c.uid; '''
datamart = pd.read_sql(query, connection,parse_dates=['first_commit_ts','first_view_ts'])
datamart.to_sql('datamart',connection,if_exists='replace')

print("Датафрейм:")
print(datamart.info())

# Создание датафрейма test
test= datamart.dropna().copy()
# control = datamart[datamart.first_view_ts.isna()].copy()
# control.first_view_ts = test.first_view_ts.mean()
test.to_sql('test',connection,if_exists='replace')
# control.to_sql('control',connection,if_exists='replace')

print("Датафрейм test:")
print(test.info())

# Создание датафрейма control с заменой пропущенных значений средним
control = datamart[datamart.first_view_ts.isna()].copy()
control.first_view_ts = test.first_view_ts.mean()
control.to_sql('control',connection,if_exists='replace')
print("Датафрейм control:")
print(control.info())

# Сохранение датафреймов в базе данных
test = pd.io.sql.read_sql('SELECT * FROM test',connection) 
control = pd.io.sql.read_sql('SELECT * FROM test',connection) 

# Закрытие соединения с базой данных
connection.close()


Датафрейм:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              140 non-null    object        
 1   labname          140 non-null    object        
 2   first_commit_ts  140 non-null    datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 4.5+ KB
None
Датафрейм test:
<class 'pandas.core.frame.DataFrame'>
Index: 59 entries, 3 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              59 non-null     object        
 1   labname          59 non-null     object        
 2   first_commit_ts  59 non-null     datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 2.3+ KB
None
Датафре